##### Copyright 2020 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License").
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# TFDV and Apache Beam sandbox

In [ ]:
%pip install tfx

In [1]:
import logging
import re

import pyarrow as pa

from typing import List, Optional, Text, Union, Dict, Iterable

import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.runners import DataflowRunner

from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

from apache_beam.io.gcp.internal.clients import bigquery

#from tensorflow_data_validation import types

from jinja2 import Template

import google.auth

In [ ]:
def generate_sampling_query(source_table_name, num_lots, lots):
  """Prepares the data sampling query."""

  sampling_query_template = """
       SELECT *
       FROM 
           `{{ source_table }}` AS cover
       WHERE 
       MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), {{ num_lots }}) IN ({{ lots }})
       """
  query = Template(sampling_query_template).render(
      source_table=source_table_name, num_lots=num_lots, lots=str(lots)[1:-1])

  return query


In [ ]:
source_table_name = 'mlops-dev-env.covertype_dataset.covertype'
num_lots = 100
lots = [1]

query = generate_sampling_query(source_table_name, num_lots, lots)

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

query_job = client.query('SELECT * FROM ({}) LIMIT 0'.format(query))
results = query_job.result()

In [ ]:
schema = {field.name: field.field_type for field in results.schema}
schema

In [12]:
import tensorflow_data_validation as tfdv
import os

In [17]:
output_path = 'gs://hostedkfp-default-36un4wco1q/tfdv/output'
#output_path = '/home/jupyter/output'
 
_STATS_FILENAME='stats.pb'
_ANOMALIES_FILENAME='anomalies.pbtxt'

stats_output_path = os.path.join(output_path, _STATS_FILENAME)
anomalies_output_path = os.path.join(output_path, _ANOMALIES_FILENAME)
 

In [18]:
stats = tfdv.load_statistics(stats_output_path)

In [19]:
tfdv.visualize_statistics(stats)

In [16]:
stats

datasets {
  num_examples: 2460
  features {
    type: STRING
    string_stats {
      common_stats {
        num_non_missing: 2460
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 246.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 246.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 246.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 246.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 246.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 246.0
          }
          buckets {
            low_value: 

In [41]:
anomalies = tfdv.load_anomalies_text(anomalies_output_path)

In [42]:
tfdv.display_anomalies(anomalies)

In [1]:
import pyarrow as pa


In [22]:
obj = [{'c1': [1], 'c2': [3.0]}, {'c1': [2], 'c2': [6.0]}]
struct_array = pa.array(obj)

In [23]:
field_names = [f.name for f in list(struct_array.type)]

In [24]:
field_names

['c1', 'c2']

In [25]:
value_arrays = struct_array.flatten()

In [26]:
value_arrays

 [
   [
     1
   ],
   [
     2
   ]
 ],
 [
   [
     3
   ],
   [
     6
   ]
 ]]

In [27]:
ar1 = value_arrays[1]

In [28]:
ar1.type

ListType(list<item: double>)

In [31]:
obj = [[1]]
pa.array(obj)

[
  [
    1
  ]
]